<br>
<a href="https://www.nvidia.com/en-us/training/">
    <div style="width: 55%; background-color: white; margin-top: 50px;">
    <img src="https://dli-lms.s3.amazonaws.com/assets/general/nvidia-logo.png"
         width="400"
         height="186"
         style="margin: 0px -25px -5px; width: 300px"/>
</a>
<h1 style="line-height: 1.4;"><font color="#76b900"><b>Building Agentic AI Applications with LLMs</h1>
<h2><b>Exercise 3:</b> Implementing Our Persona System In LangGraph</h2>
<br>

In the prior section, you explored using LangGraph to orchestrate nodes and edges for a basic agentic workflow. Now, let’s revisit our little persona agent problem from Section 1 involving the teacher, the student, and the parent. We now have a structured outputs and LangGraph experience, so maybe we can orchestrate our system using this new abstraction? You’ll see how to set up each persona’s data, create a unified prompt format, generate structured JSON responses, and chain these agents together in a state graph.

### **Learning Objectives:**
**In this notebook, we will:**

- Get some practice with LangGraph and get comfortable with its state management abstractions.
- Implement a proper attempt at routing based on the LLM's direction (in contrast to our failed ReAct attempts in Notebook 2t).

In [ ]:
from langchain_nvidia import ChatNVIDIA

llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct", base_url="http://llm_client:9000/v1")

<hr><br>

### **Part 1:** Pulling In Our Personas

You may remember both our bespoke system in base Python and our streamlined system in CrewAI, so let's just pool those specifications together to create our couple of personas:

In [ ]:
teacher_args = dict(
    role="John Doe (Teacher)",
    backstory=(
        "You are a computer science teacher in high school holding office hours, and you have a meeting."
        " This is the middle of the semester, and various students have various discussion topics across your classes."
        " You are having a meeting right now. Please engage with the students and help their parent."
    ), 
    directive="You are having a meeting right now. Please engage with the other speakers and help them out with their concerns.",
)

student1_args = dict(
    role="Jensen (Student)",
    backstory="You are taking Dr. Doe's intro to algorithms course and are struggling with some of the homework problems.", 
    directive="Meet with your teacher to help you understand class material. Respond and ask directed questions, contributing to discussion.",
)

student2_args = dict(
    role="Albert (Student)",
    backstory="You are taking Dr. Doe's intro to algorithms course and are struggling with some of the homework problems.", 
    directive="Meet with your teacher to help you understand class material. Respond and ask directed questions, contributing to discussion.",
)

parent_args = dict(
    role="Sally (Parent)",
    backstory="You are here with your kids, who are students in the teacher's class.", 
    directive="Meet with your kids and the teacher to help support the students and see what you can do better.",
)

agent_unique_spec_dict = {args.get("role"):args for args in [teacher_args, student1_args, student2_args, parent_args]}

<br>

Now, let’s build a `ChatPromptTemplate` that is flexible enough to apply to each persona. You’ll see placeholders for:
- `{role}`, `{backstory}`, and `{directive}` from our agent specs.
- A space for the final `schema_hint`, which we will use to help route our system/
- The message placeholder which will contain our conversation messages so far.



In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from course_utils import SCHEMA_HINT  ## <- Convenience method to get schema hint template

## Define the structured prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You are {role}. {backstory}"
        "\nThe following people are in the room: {role_options}."
        " {directive}\n" f"{SCHEMA_HINT}"
    )),
    ("placeholder", "{messages}"),
])

<hr><br>

### **Step 2:** Defining Our Response Schemas

Using a similar logic as in Notebook 2, we can endow our systems with structured output to help us not only get a natural language response, but also generate pathing variables which we can then use to route our conversation.

Guiding the decoding based on the legal pathways of the current state is a bit hard to manage, but can be controlled by tweaking the schema that is sent in to the LLM endpoint. A convenience method `get_finite_schema` is defined below.

In [ ]:
from pydantic import BaseModel, Field
from typing import Any, Dict, List, Literal

## Definition of Desired Schema
class AgentResponse(BaseModel):
    """
    Defines the structured response of an agent in the conversation.
    Ensures that each agent response includes the speaker's identity,
    a list of response messages, and a defined routing option.
    """
    speaker: Literal["option1", "option2"] = Field(description="Who are you responding as?")
    response: List[str] = Field(description="Response to contribute to the conversation")
    route: Literal["option1", "option2"] = Field(description="A choice of the next person")

    @classmethod
    def get_default(cls):
        return cls(speaker="option1", response=[], route="option1")
    
    @classmethod
    def get_finite_schema(cls, key_options: Dict[str, List[str]]) -> Dict[str, Any]:
        """
        Dynamically modifies the schema to adjust the possible routing options.
        This is useful for ensuring the model respects dynamic conversation flows.
        """
        schema = cls.model_json_schema()
        for key, options in key_options.items():
            if "enum" in schema["properties"].get(key, {}):
                schema["properties"][key]["enum"] = options
            if "items" in schema["properties"].get(key, {}):
                schema["properties"][key]["items"] = {'enum': options, 'type': 'string'}
        return schema

role_options = list(agent_unique_spec_dict.keys()) + ["End"]
schema_hint = AgentResponse.get_finite_schema({"speaker": role_options[:1], "route": role_options})
schema_hint

<br>

And just like that, we now have both the local and global specification needed to populate our prompt template. These will serve as our arguments for constructing our purpose-build **Agent** class.

In [ ]:
## Shared parameters across agents
shared_args = dict(
    llm=llm, 
    schema=AgentResponse.get_default(), 
    schema_hint=schema_hint, 
    prompt=prompt, 
    routes=role_options, 
    roles=role_options
)

## Initialize agent specifications with shared parameters
agent_spec_dict = {
    role: {**unique_specs, **shared_args} 
    for role, unique_specs in agent_unique_spec_dict.items()
}

<hr><br>

### **Step 3:** Defining Our Agent Class

To help keep the complexity out of our final orchestration graph, we can implement some stateful agents just like the ones found in our CrewAI example. 

- To stick with our theory-guided approach towards agentics, the interface leading to and from the LLM are called `_convert_to_local` and `_convert_to_global`, respectively. If you check them out, you'll notice they look strikingly familiar.
- You'll notice that in `_get_llm`, we parameterize the `get_finite_method` with the classes that we may want our LLM to choose between (or have no choice over). Note that this is not an officially-supported method across LangChain/LangGraph, and is just there to simplify our codebase.
- We didn't really make noise about it last time, but you'll notice we're calling the llm with `.invoke`. It sure would be weird if it started streaming the outputs when we actually used the class...

In [ ]:
from langchain_core.output_parsers import JsonOutputParser 
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field, ValidationError
from typing import List, Literal, Dict, Any
import ast


class Agent:
    """
    Represents an interactive agent that responds to messages in a structured format.
    Each agent is initialized with an LLM and a predefined prompt to ensure consistency.
    """
    
    def __init__(self, llm, prompt, role, routes, schema=None, **kwargs):
        self.llm = llm
        self.role = role
        self.prompt = prompt
        self.routes = routes
        self.schema = schema
        ## Let's funnel all of our prompt arguments into the default_kwargs
        self.default_kwargs = {**kwargs, "role": self.role, "role_options": "/".join(self.routes)}

    def __call__(self, config=None, **kwargs):
        """
        Calls the agent with a given message and retrieves a structured response.
        """
        kwarg_pool = {**self.default_kwargs, **kwargs}
        global_inputs = kwarg_pool.get("messages")
        local_inputs = self._convert_to_local(**{**kwarg_pool, "messages": global_inputs})
        local_outputs = self._invoke_llm(**{**kwarg_pool, "messages": local_inputs})
        global_outputs = self._convert_to_global(**{**kwarg_pool, "messages": local_outputs})
        return global_outputs
    
    def _get_llm(self, **kwargs):
        """
        Retrieves the LLM with the appropriate structured output schema if available.
        """
        if self.schema:
            if hasattr(self.schema, "get_finite_schema"):
                current_schema = self.schema.get_finite_schema({
                    "speaker": [self.role], 
                    "route": [r for r in self.routes if r != self.role],
                })
            else: 
                current_schema = getattr(self.schema, "model_json_schema", lambda: self.schema)()
            return self.llm.with_structured_output(schema=current_schema, strict=True)
        return self.llm
    
    def _invoke_llm(self, config=None, **kwargs):
        """
        Invokes the LLM with the constructed prompt and provided configuration.
        Adds debugging support to inspect inputs.
        """
        llm_inputs = self.prompt.invoke(kwargs)
        # print("\nINPUT TO LLM:", "\n".join(repr(m) for m in llm_inputs.messages[1:]))
        llm_output = self._get_llm(**kwargs).invoke(llm_inputs, config=config)
        # print("\nOUTPUT FROM LLM:", repr(llm_output))
        return [llm_output]

    def _convert_to_local(self, messages: List[tuple], **kwargs) -> List[tuple]:
        """
        Converts input messages into a format suitable for processing by the LLM.
        """
        dict_messages = self._convert_to_global(messages)
        roled_messages = [(v.get("speaker"), v.get("response")) for v in dict_messages]
        local_messages = list((
            "ai" if speaker == self.role else "user", 
            f"[{speaker}] " + '\n'.join(content) if isinstance(content, list) else content
        ) for speaker, content in roled_messages)
        return local_messages
    
    def _convert_to_global(self, messages, **kwargs) -> Dict[str, Any]:
        """
        Converts various response formats into a structured dictionary format.
        Handles potential edge cases, including string responses.
        """
        outputs = []
        for msg in messages:
            if isinstance(msg, tuple):
                outputs += [{"speaker": msg[0], "response": msg[1]}]
            elif isinstance(msg, dict):
                outputs += [msg]
            elif isinstance(msg, str) or hasattr(msg, "content"):   
                try:
                    outputs += [ast.literal_eval(getattr(msg, "content", msg))]  ## Strongly assumes good format
                except (SyntaxError, ValueError) as e:
                    print(f"Error parsing response: {e}")
                    outputs += [{"speaker": "Unknown", "response": ["Error encountered"], "route": "End"}]
            else:
                print(f"Encountered Unknown Message Type: {e}")
                outputs += [{"speaker": "Unknown", "response": ["Error encountered"], "route": "End"}]
        return outputs
    

## Initialize conversation
messages = [("Jensen (Student)", "Hello! How's it going?")]

## Start with the first agent
teacher_agent = Agent(**list(agent_spec_dict.values())[0])
response = teacher_agent(messages=messages)[0]
print(response)

## TODO: Route to the next agent based on response

## TODO: Continue the conversation for one more turn


<details><summary><b>Hint</b></summary>

Make sure to take advantage of your message buffer. Maybe the first step is to add the generated message to the buffer? From there, we just need to route to the appropriate agent based on the response...

</details>


<details><summary><b>Solution</b></summary>

```python
## Start with the first agent
teacher_agent = Agent(**list(agent_spec_dict.values())[0])
response = teacher_agent(messages=messages)[0]
print(response)
messages.append((response.get("speaker"), response.get("response")))

## TODO: Route to the next agent based on response
next_agent = Agent(**agent_spec_dict.get(response.get("route"), {}))
response = next_agent(messages=messages)[0]
print(response)
messages.append((response.get("speaker"), response.get("response")))

## TODO: Continue the conversation
next_agent = Agent(**agent_spec_dict.get(response.get("route"), {}))
response = next_agent(messages=messages)[0]
print(response)
```

</details>

<hr><br>

### **Part 4:** Putting It All Together

Now that we have all of these components, we can integrate them together to make our own agent system fit for the use-case. Much in the same way as before, we can do all of this with only a single agent abstraction, but each agent can just have their own class instance. As an exercise, see if you can't construct the agent class without looking at the solution!

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.constants import START, END
from langgraph.graph import StateGraph
from langgraph.types import interrupt, Command
from langgraph.graph.message import add_messages

from typing import Annotated, Dict, List, Optional, TypedDict
import operator

##################################################################
## Define the authoritative state system (environment) for your use-case

class State(TypedDict):
    """The Graph State for your Agent System"""
    messages: Annotated[list, add_messages] = []
    agent_dict: Dict[str, dict]
    speakers: List[str] = []  ## <- use this to keep track of routing/enqueueing

##################################################################
## Define the operations (Nodes) that can happen on your environment

def agent(state: State):
    """Edge option where transition is generated at runtime"""
    agent_dict = state.get("agent_dict")
    current_speaker = state.get("speakers")[-1]
    ## TODO: If a speaker is retrieved properly, construct the agent connector,
    ## generate the response, and route to the appropriate next speaker.
    if current_speaker in agent_dict:
        current_agent = Agent(**agent_dict[current_speaker])
        response = current_agent(**state)[0]
        return Command(update={
            "messages": [("ai", str(response))], 
            "speakers": [response.get("route")],
        }, goto="agent")

##################################################################
## Define the system that organizes your nodes (and maybe edges)

builder = StateGraph(State)
builder.add_node("agent", agent)
builder.add_edge(START, "agent")  ## A start node is always necessary

In [ ]:
from course_utils import stream_from_app
from functools import partial
import uuid

checkpointer = MemorySaver()
app = builder.compile(checkpointer=checkpointer)
config = {"configurable": {"thread_id": uuid.uuid4()}, "recursion_limit": 1000}
app_stream = partial(app.stream, config=config)

## We can stream over it until an interrupt is received

initial_inputs = {"messages": [], "agent_dict": agent_spec_dict, "speakers": list(agent_spec_dict.keys())[:1]}
for token in stream_from_app(app_stream, input_buffer=[initial_inputs], verbose=False, debug=False):
    if token == "[Agent]: ":
        print("\n", flush=True)
    print(token, end="", flush=True)
    

<hr><br>

### **Part 5:** Reflecting On This Exercise

This may very well be the hardest system you've implemented today. We had to conform to the LangGraph logic, define some custom non-intuitive utilities, and validate our decisions every step of the way. By now you probably can see that this is **significantly harder** than our approach in CrewAI, and that's ok! Part of the appeal of LangGraph is that it is, in fact, a highly-customizable solution which can scale to production with relative ease and a high amount of observability/control defined at any level.

You may recall that the LangGraph ReAct loop didn't work very well out-of-the-box from our model, though it was implemented in a sound way such that a different model will actually work better. The fact that we could completely undercut this abstraction and make it exactly like we wanted to, as we did here, is the real feature to appreciate. And also, we needed to offer some practice before the assessment, so... it works out!

**In the next section, get ready to try out the assessment to see if you can make an interesting research agent based on the tools we've discussed today! (But before then, the warm-up may be of interest)**

<a href="https://www.nvidia.com/en-us/training/">
    <div style="width: 55%; background-color: white; margin-top: 50px;">
    <img src="https://dli-lms.s3.amazonaws.com/assets/general/nvidia-logo.png"
         width="400"
         height="186"
         style="margin: 0px -25px -5px; width: 300px"/>